In [ ]:
!pip install -U "trl>=0.8.6" "transformers>=4.44" "peft>=0.12" "bitsandbytes>=0.43" "accelerate>=1.0"

  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.0
    Uninstalling bitsandbytes-0.41.0:
      Successfully uninstalled bitsandbytes-0.41.0


In [ ]:

!pip uninstall -y bitsandbytes accelerate peft transformers trl
!pip install bitsandbytes
!pip install --upgrade accelerate peft trl transformers

Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.1
Uninstalling peft-0.18.1:
  Successfully uninstalled peft-0.18.1
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: trl 0.26.2
Uninstalling trl-0.26.2:
  Successfully uninstalled trl-0.26.2
  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
  Using cached peft-0.18.1-py3-none-any.whl.metadata (14 kB)
  Using cached trl-0.26.2-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%%capture
# Uninstall conflicting versions first
!pip uninstall -y trl transformers accelerate bitsandbytes peft datasets

# Install Unsloth and its dependencies correctly
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install specific versions for compatibility with Unsloth 2026.1.2
# Unsloth requires trl <= 0.24.0 and >= 0.18.2
!pip install --no-deps "trl==0.24.0" "transformers==4.57.3" "peft==0.18.1" "accelerate==1.12.0" "bitsandbytes==0.49.1" "datasets==4.3.0"


In [ ]:
import json
import glob
import os

# --- CONFIGURARE ---
# Aici pui calea către folderul unde ai încărcat fișierele JSON
# Dacă ai folosit codul anterior, probabil este aceasta:
folder_path = "/content/drive/MyDrive/ToateBolile/"

# --- SCRIPTUL DE NUMĂRARE ---
json_files = glob.glob(os.path.join(folder_path, "*.json"))
total_conversations = 0

print(f"🔍 Analizez fișierele din: {folder_path}\n")
print(f"{'NUME FIȘIER':<40} | {'NR. CONVERSAȚII'}")
print("-" * 60)

for file_path in json_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            # Verificăm dacă fișierul este o listă (formatul standard)
            if isinstance(data, list):
                count = len(data)
                print(f"{os.path.basename(file_path):<40} | {count}")
                total_conversations += count
            else:
                print(f"{os.path.basename(file_path):<40} | 0 ⚠️ (Nu este o listă)")

    except Exception as e:
        print(f"{os.path.basename(file_path):<40} | EROARE: {e}")

print("-" * 60)
print(f"✅ TOTAL GENERAL: {total_conversations} conversații")
print("-" * 60)

# Sfat pentru antrenare bazat pe numărul total
if total_conversations < 100:
    print("⚠️ Sfat: Ai destul de puține date. Folosește 'num_train_epochs = 3' sau mai mult.")
elif total_conversations > 500:
    print("✅ Sfat: Ai un număr bun de date. O singură epocă ('num_train_epochs = 1') ar putea fi suficientă.")
else:
    print("ℹ️ Sfat: Număr mediu de date. Încearcă 1-2 epoci.")

🔍 Analizez fișierele din: /content/drive/MyDrive/ToateBolile/

NUME FIȘIER                              | NR. CONVERSAȚII
------------------------------------------------------------
pulp_necrosis.json                       | 25
periodontal_abscess.json                 | 13
simple_caries.json                       | 23
acute_apical_abscess.json                | 39
acute_total_pulpitis.json                | 28
chronic_apical_periodontitis.json        | 32
denture_related_pain.json                | 20
otitis.json                              | 20
pericoronitis.json                       | 53
peritonsillar_abscess.json               | 15
reversible_pulpitis.json                 | 44
sialolithiasis.json                      | 20
tmj_pain.json                            | 20
trigeminal_neuralgia.json                | 20
acute_apical_periodontitis.json          | 13
------------------------------------------------------------
✅ TOTAL GENERAL: 385 conversații
---------------------------------

new fine-tunning

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, train_on_responses_only
import os

# Dezactivăm WANDB pentru a nu cere login
os.environ["WANDB_DISABLED"] = "true"

# Configurații de bază
max_seq_length = 2048
dtype = None # Revert to None to let Unsloth manage dtype for 4-bit loading
load_in_4bit = True

print("--- 1. Incarcare Model si Tokenizer ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Configurăm LoRA (Adaptoarele)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# --- 2. Configurare Template Llama-3 (CRITIC) ---
# Aceasta funcție setează automat pad_token, eos_token și padding_side corect
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role": "role", "content": "content", "user": "user", "assistant": "assistant"},
)

# --- 3. Formatare Date ---
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = []
    for convo in convos:
        # apply_chat_template face toată magia: adaugă header-ele și EOS la final
        texts.append(tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False))
    return { "text" : texts }

# Încărcăm datele
print("--- 4. Incarcare si Impartire Dataset ---")
full_dataset = load_dataset("json", data_files="/content/drive/MyDrive/ToateBolile/*.json", split="train")

# Aplicăm formatarea
full_dataset = full_dataset.map(formatting_prompts_func, batched = True)

# Împărțim dataset-ul: 90% Antrenament, 10% Evaluare (Test)
# La 385 conversații => ~346 antrenament, ~39 testare
dataset_split = full_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"Total: {len(full_dataset)}. Train: {len(train_dataset)}, Eval: {len(eval_dataset)}")

# --- 5. Configurare Trainer ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset, # Adăugăm setul de evaluare
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,

        # Folosim Epoci în loc de max_steps pentru că avem dataset mare acum
        num_train_epochs = 3,

        # Configurare Evaluare și Salvare
        eval_strategy = "steps",
        eval_steps = 50,         # Verificăm loss-ul pe setul de test la fiecare 50 pași
        save_strategy = "steps",
        save_steps = 50,         # Salvăm checkpoint la fiecare 50 pași
        save_total_limit = 3,  # ← ADAUGĂ ASTA
        load_best_model_at_end = True,  # ← Încarcă best checkpoint la final
        metric_for_best_model = "eval_loss",

        learning_rate = 2e-4,
        fp16 = True, # Force FP16 as BFloat16 is not supported on T4
        bf16 = False, # Explicitly disable BFloat16
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

# --- 6. MASKING (PRO TIP) ---
# Această linie transformă trainer-ul să învețe DOAR răspunsurile Asistentului.
# Ignoră loss-ul pe întrebările userului. Face antrenamentul mult mai eficient.
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)
print("--- 🔍 VERIFICARE DATE ---")
print(f"\n📊 Statistici:")
print(f"  • Total conversații: {len(full_dataset)}")
print(f"  • Train: {len(train_dataset)} ({len(train_dataset)/len(full_dataset)*100:.1f}%)")
print(f"  • Eval: {len(eval_dataset)} ({len(eval_dataset)/len(full_dataset)*100:.1f}%)")
print(f"  • Batch efectiv: {2 * 4}")
print(f"  • Steps/epoch: ~{len(train_dataset) // (2*4)}")
print(f"  • Total steps estimate: ~{3 * (len(train_dataset) // (2*4))}")

print("\n🔎 Exemplu conversație formatată:")
print("="*60)
print(train_dataset[0]["text"][:500])
print("...")
print("="*60)

input("\n⏸️  Apasă ENTER pentru a începe antrenarea...")  # Pauză pentru verificare

print("--- 7. Incepem Antrenarea ---")
# Va afișa și 'Validation Loss' acum
trainer_stats = trainer.train()

print("--- 8. Salvare Model Final ---")
model.save_pretrained("llama3_dental_patient_v1")
tokenizer.save_pretrained("llama3_dental_patient_v1")
print("Gata! Modelul a fost salvat in folderul 'llama3_dental_patient_v1'.")

print("--- 9. Test Rapid ---")
FastLanguageModel.for_inference(model)  # Setează modelul în modul inferență

test_messages = [
    {"role": "user", "content": "Ce este caria dentară?"}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

print("\n🤖 Răspuns model:")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
--- 1. Incarcare Model si Tokenizer ---
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2026.1.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


--- 4. Incarcare si Impartire Dataset ---


Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Total: 385. Train: 346, Eval: 39


...

--- 7. Incepem Antrenarea ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 346 | Num Epochs = 3 | Total steps = 132
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.237800,0.668619
100,0.269100,0.615345


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
pip install datasets==4.3.0

  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
Using cached datasets-4.3.0-py3-none-any.whl (506 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.2
    Uninstalling datasets-4.4.2:
      Successfully uninstalled datasets-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2026.1.2 requires trl!=0.19.0,<=0.24.0,>=0.18.2, but you have trl 0.26.2 which is incompatible.


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

MessageError: Error: credential propagation was unsuccessful